In [1]:
pip install yfinance pandas numpy scikit-learn matplotlib seaborn gym torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
import yfinance as yf
import pandas as pd

In [3]:
# Define tickers and date range
tickers = ['AAPL', 'RIOT', 'PLUG']
start_date = '2014-01-01'
end_date = '2024-12-31'

# Dictionary to store individual DataFrames
adj_close_data = {}

# Loop through each ticker
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)

    # Keep only 'Date' and 'Adj Close'
    df = data[['Adj Close']].reset_index()
    df.rename(columns={'Adj Close': f'{ticker}_AdjClose'}, inplace=True)

    adj_close_data[ticker] = df

# Example: show the first few rows of PLUG
# print(adj_close_data['PLUG'].head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Clean Data next

In [4]:
for ticker, df in adj_close_data.items():
    # Drop rows with missing values
    df.dropna(inplace=True)

    # Convert 'Date' to datetime (in case it isn't already)
    df['Date'] = pd.to_datetime(df['Date'])

    # Sort chronologically
    df.sort_values(by='Date', inplace=True)

    # Reset index
    df.reset_index(drop=True, inplace=True)

    # Store back the cleaned DataFrame
    adj_close_data[ticker] = df

# Preview cleaned data
# print(adj_close_data['PLUG'].head())


compute technical indicators

In [5]:
# Loop through each symbol and compute indicators
for ticker, df in adj_close_data.items():
    # Use the adjusted close column
    price_col = f'{ticker}_AdjClose'

    # 20-day Simple Moving Average
    df['SMA_20'] = df[price_col].rolling(window=20).mean()

    # 10-day Momentum
    df['Momentum_10'] = df[price_col] - df[price_col].shift(10)

    # 10-day Volatility (standard deviation of past 10 days)
    df['Volatility_10'] = df[price_col].rolling(window=10).std()

    # Drop rows with NaN values resulting from rolling calculations
    df.dropna(inplace=True)

    # Reset index again after dropping rows
    df.reset_index(drop=True, inplace=True)

# Example: display indicators for TSLA
# print(adj_close_data['TSLA'][['Date', 'TSLA_AdjClose', 'SMA_20', 'Momentum_10', 'Volatility_10']].head())


Label Creation Code

In [6]:
for ticker, df in adj_close_data.items():
    price_col = f'{ticker}_AdjClose'

    # Compute forward 5-day return
    df['Forward_5d_Return'] = (df[price_col].shift(-5) - df[price_col]) / df[price_col]

    # Assign labels based on thresholds
    def label_return(r):
        if r > 0.02:
            return 1
        elif r < -0.02:
            return -1
        else:
            return 0

    df['Label'] = df['Forward_5d_Return'].apply(label_return)

    # Drop final rows with NaN from shift(-5)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

# Example: show labeled data for AAPL
# print(adj_close_data['AAPL'][['Date', 'AAPL_AdjClose', 'Forward_5d_Return', 'Label']].head(10))


Training/Testing Set Split

In [7]:
# Dictionary to hold training and testing sets
train_test_split = {}

for ticker, df in adj_close_data.items():
    # Ensure 'Date' is datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Split the data
    train_df = df[df['Date'] < '2020-01-01'].copy()
    test_df = df[df['Date'] >= '2020-01-01'].copy()

    # Store in dictionary
    train_test_split[ticker] = {
        'train': train_df,
        'test': test_df
    }

# Example: view sizes of each split for TSLA
# print("TSLA Training set size:", len(train_test_split['TSLA']['train']))
# print("TSLA Testing set size:", len(train_test_split['TSLA']['test']))


Baseline Model #1: Random Guessing Model

In [8]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [9]:
random_guess_results = {}
initial_capital = 100000

for ticker, splits in train_test_split.items():
    test_df = splits['test'].copy()

    # Generate random predictions
    test_df['Predicted_Action'] = np.random.choice([-1, 0, 1], size=len(test_df))

    # Simulate portfolio
    in_position = False
    entry_price = 0
    portfolio_value = float(initial_capital)
    portfolio_history = []

    for i, row in test_df.iterrows():
        action = int(row['Predicted_Action'])
        price = float(row[f'{ticker}_AdjClose'])

        if action == 1 and not in_position:
            entry_price = price
            in_position = True

        elif action == -1 and in_position:
            return_pct = (price - entry_price) / entry_price
            portfolio_value *= (1 + return_pct)
            in_position = False

        portfolio_history.append(portfolio_value)

    # If still in a position at the end, close it
    if in_position:
        final_price = float(test_df[f'{ticker}_AdjClose'].iloc[-1])
        return_pct = (final_price - entry_price) / entry_price
        portfolio_value *= (1 + return_pct)

    # Metrics
    accuracy = accuracy_score(test_df['Label'], test_df['Predicted_Action'])
    conf_matrix = confusion_matrix(test_df['Label'], test_df['Predicted_Action'], labels=[-1, 0, 1])
    class_report = classification_report(test_df['Label'], test_df['Predicted_Action'], labels=[-1, 0, 1], output_dict=True)

    random_guess_results[ticker] = {
        'final_portfolio_value': portfolio_value,
        'return_pct': (portfolio_value - initial_capital) / initial_capital,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'classification_report': class_report
    }

    print(f"\n--- {ticker} [Random Guessing] ---")
    print(f"Final Portfolio Value: ${portfolio_value:,.2f}")
    print(f"Total Return: {(portfolio_value - initial_capital) / initial_capital:.2%}")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:\n", conf_matrix)



/tmp/ipython-input-9-1239231236.py:17: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  action = int(row['Predicted_Action'])
/tmp/ipython-input-9-1239231236.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(row[f'{ticker}_AdjClose'])
/tmp/ipython-input-9-1239231236.py:33: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  final_price = float(test_df[f'{ticker}_AdjClose'].iloc[-1])
/tmp/ipython-input-9-1239231236.py:17: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  action = int(row['Predicted_Action'])
/tmp/ipython-input-9-1239231236.py:18: FutureWarning: Calling float on a single element Series 


--- AAPL [Random Guessing] ---
Final Portfolio Value: $214,930.19
Total Return: 114.93%
Accuracy: 0.3578
Confusion Matrix:
 [[111  93  92]
 [163 181 164]
 [153 139 156]]


/tmp/ipython-input-9-1239231236.py:33: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  final_price = float(test_df[f'{ticker}_AdjClose'].iloc[-1])
/tmp/ipython-input-9-1239231236.py:17: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  action = int(row['Predicted_Action'])
/tmp/ipython-input-9-1239231236.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(row[f'{ticker}_AdjClose'])



--- RIOT [Random Guessing] ---
Final Portfolio Value: $313,744.78
Total Return: 213.74%
Accuracy: 0.3307
Confusion Matrix:
 [[181 184 191]
 [ 46  47  47]
 [195 175 186]]

--- PLUG [Random Guessing] ---
Final Portfolio Value: $49,288.71
Total Return: -50.71%
Accuracy: 0.3187
Confusion Matrix:
 [[168 211 192]
 [ 48  63  54]
 [183 165 168]]


Baseline Model #2: Buy and Hold Strategy

In [10]:
buy_hold_results = {}
initial_capital = 100000

for ticker, splits in train_test_split.items():
    test_df = splits['test'].copy()

    # Get adjusted close prices as floats
    entry_price = float(test_df[f'{ticker}_AdjClose'].iloc[0])
    exit_price = float(test_df[f'{ticker}_AdjClose'].iloc[-1])

    # Portfolio value after holding through test period
    return_pct = (exit_price - entry_price) / entry_price
    final_value = initial_capital * (1 + return_pct)

    # Simulate constant strategy:
    # Buy at start (1), hold (0) throughout, sell at end (-1)
    actions = [1] + [0] * (len(test_df) - 2) + [-1]
    test_df['Predicted_Action'] = actions

    # Evaluate prediction vs true labels
    accuracy = accuracy_score(test_df['Label'], test_df['Predicted_Action'])
    conf_matrix = confusion_matrix(test_df['Label'], test_df['Predicted_Action'], labels=[-1, 0, 1])
    class_report = classification_report(test_df['Label'], test_df['Predicted_Action'], labels=[-1, 0, 1], output_dict=True)

    buy_hold_results[ticker] = {
        'final_portfolio_value': final_value,
        'return_pct': return_pct,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'classification_report': class_report
    }

    print(f"\n--- {ticker} [Buy and Hold] ---")
    print(f"Final Portfolio Value: ${final_value:,.2f}")
    print(f"Total Return: {return_pct:.2%}")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:\n", conf_matrix)





--- AAPL [Buy and Hold] ---
Final Portfolio Value: $349,593.26
Total Return: 249.59%
Accuracy: 0.4058
Confusion Matrix:
 [[  0 296   0]
 [  1 507   0]
 [  0 447   1]]

--- RIOT [Buy and Hold] ---
Final Portfolio Value: $946,721.30
Total Return: 846.72%
Accuracy: 0.1134
Confusion Matrix:
 [[  1 555   0]
 [  0 140   0]
 [  0 555   1]]

--- PLUG [Buy and Hold] ---
Final Portfolio Value: $79,012.34
Total Return: -20.99%
Accuracy: 0.1334
Confusion Matrix:
 [[  1 570   0]
 [  0 165   0]
 [  0 515   1]]


/tmp/ipython-input-10-1867281624.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  entry_price = float(test_df[f'{ticker}_AdjClose'].iloc[0])
/tmp/ipython-input-10-1867281624.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  exit_price = float(test_df[f'{ticker}_AdjClose'].iloc[-1])
/tmp/ipython-input-10-1867281624.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  entry_price = float(test_df[f'{ticker}_AdjClose'].iloc[0])
/tmp/ipython-input-10-1867281624.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  exit_price = float(test_df[f'{ticker}_AdjClose'].iloc[-1])
/tmp/ipython-input-10-186728

Baseline Model #3 - Random Forest Classifier

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [12]:
# Flatten column names for all tickers and both splits
for ticker, splits in train_test_split.items():
    for split_name in ['train', 'test']:
        df = splits[split_name]
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join(col).strip('_') for col in df.columns.values]
        splits[split_name] = df  # reassign flattened DataFrame


In [13]:

random_forest_results = {}
initial_capital = 100000
features = ['SMA_20', 'Momentum_10', 'Volatility_10']

for ticker, splits in train_test_split.items():
    train_df = splits['train'].copy()
    test_df = splits['test'].copy()

    # Drop rows with missing values in features or label
    train_df.dropna(subset=features + ['Label'], inplace=True)
    test_df.dropna(subset=features + ['Label'], inplace=True)

    # Prepare feature matrix and target vector
    X_train = train_df[features]
    y_train = train_df['Label']
    X_test = test_df[features]
    y_test = test_df['Label']

    # Train Random Forest
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Predict trading actions
    test_df['Predicted_Action'] = clf.predict(X_test)

    # 🔍 Find adjusted close column dynamically
    adj_close_col_candidates = [col for col in test_df.columns if 'AdjClose' in col and ticker in col]
    if not adj_close_col_candidates:
        raise KeyError(f"No adjusted close column found for {ticker}")
    adj_close_col = adj_close_col_candidates[0]

    # Simulate portfolio
    in_position = False
    entry_price = 0
    portfolio_value = float(initial_capital)
    portfolio_history = []

    for _, row in test_df.iterrows():
        action = int(row['Predicted_Action'])
        price = float(row[adj_close_col])

        if action == 1 and not in_position:
            entry_price = price
            in_position = True

        elif action == -1 and in_position:
            return_pct = (price - entry_price) / entry_price
            portfolio_value *= (1 + return_pct)
            in_position = False

        portfolio_history.append(portfolio_value)

    # Close any open position at end
    if in_position:
        final_price = float(test_df[adj_close_col].iloc[-1])
        return_pct = (final_price - entry_price) / entry_price
        portfolio_value *= (1 + return_pct)

    # Evaluation metrics
    accuracy = accuracy_score(test_df['Label'], test_df['Predicted_Action'])
    conf_matrix = confusion_matrix(test_df['Label'], test_df['Predicted_Action'], labels=[-1, 0, 1])
    class_report = classification_report(test_df['Label'], test_df['Predicted_Action'], labels=[-1, 0, 1], output_dict=True)

    # Store results
    random_forest_results[ticker] = {
        'final_portfolio_value': portfolio_value,
        'return_pct': (portfolio_value - initial_capital) / initial_capital,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'classification_report': class_report
    }

    # Print summary
    print(f"\n--- {ticker} [Random Forest] ---")
    print(f"Final Portfolio Value: ${portfolio_value:,.2f}")
    print(f"Total Return: {(portfolio_value - initial_capital) / initial_capital:.2%}")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:\n", conf_matrix)



--- AAPL [Random Forest] ---
Final Portfolio Value: $456,188.94
Total Return: 356.19%
Accuracy: 0.3674
Confusion Matrix:
 [[ 19  95 182]
 [ 22 176 310]
 [ 13 170 265]]

--- RIOT [Random Forest] ---
Final Portfolio Value: $514,032.00
Total Return: 414.03%
Accuracy: 0.4529
Confusion Matrix:
 [[423  25 108]
 [110   8  22]
 [404  16 136]]

--- PLUG [Random Forest] ---
Final Portfolio Value: $126,989.76
Total Return: 26.99%
Accuracy: 0.4305
Confusion Matrix:
 [[451  24  96]
 [126   7  32]
 [415  20  81]]


**Reinforcement Learning Model**

Set up Gym-like Environment

In [21]:
!pip install gym

In [14]:
import gym
from gym import spaces
import numpy as np

class TradingEnv(gym.Env):
    def __init__(self, df, initial_balance=100000):
        super(TradingEnv, self).__init__()
        self.df = df.reset_index(drop=True)
        self.initial_balance = initial_balance
        self.current_step = 0
        self.in_position = False
        self.entry_price = 0.0
        self.balance = initial_balance

        # Observation: [SMA, Momentum, Volatility, Position]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32)
        # Actions: 0=Hold, 1=Buy, 2=Sell
        self.action_space = spaces.Discrete(3)

    def _get_state(self):
        row = self.df.loc[self.current_step]
        return np.array([
            row['SMA_20'],
            row['Momentum_10'],
            row['Volatility_10'],
            float(self.in_position)
        ], dtype=np.float32)

    def step(self, action):
        done = False
        reward = 0
        row = self.df.loc[self.current_step]
        price = row[[col for col in self.df.columns if 'AdjClose' in col][0]]

        # Execute action
        if action == 1 and not self.in_position:  # Buy
            self.entry_price = price
            self.in_position = True
        elif action == 2 and self.in_position:  # Sell
            reward = (price - self.entry_price) / self.entry_price
            self.balance *= (1 + reward)
            self.in_position = False
        elif action != 0:
            reward = -0.001  # small penalty for invalid action (e.g., buying twice)

        self.current_step += 1
        if self.current_step >= len(self.df) - 1:
            done = True
            # Liquidate open position
            if self.in_position:
                price = self.df[[col for col in self.df.columns if 'AdjClose' in col][0]].iloc[self.current_step]
                reward = (price - self.entry_price) / self.entry_price
                self.balance *= (1 + reward)
                self.in_position = False

        return self._get_state(), reward, done, {}

    def reset(self):
        self.current_step = 0
        self.in_position = False
        self.entry_price = 0.0
        self.balance = self.initial_balance
        return self._get_state()



Set up DQN Agent using PyTorch

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)

class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.95, epsilon=1.0, eps_min=0.01, eps_decay=0.995):
        self.q_net = DQN(state_dim, action_dim)
        self.target_net = DQN(state_dim, action_dim)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.memory = deque(maxlen=5000)
        self.optimizer = optim.Adam(self.q_net.parameters(), lr=lr)
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_min
        self.eps_decay = eps_decay
        self.action_dim = action_dim
        self.loss_fn = nn.MSELoss()

    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randrange(self.action_dim)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_vals = self.q_net(state)
        return torch.argmax(q_vals).item()

    def store(self, transition):
        self.memory.append(transition)

    def update(self, batch_size=32):
        if len(self.memory) < batch_size:
            return

        batch = random.sample(self.memory, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions).unsqueeze(1)
        rewards = torch.FloatTensor(rewards).unsqueeze(1)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones).unsqueeze(1)

        q_vals = self.q_net(states).gather(1, actions)
        with torch.no_grad():
            q_next = self.target_net(next_states).max(1)[0].unsqueeze(1)
            q_target = rewards + self.gamma * q_next * (1 - dones)

        loss = self.loss_fn(q_vals, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target_network(self):
        self.target_net.load_state_dict(self.q_net.state_dict())

    def decay_epsilon(self):
        self.epsilon = max(self.eps_min, self.epsilon * self.eps_decay)


RL Testing Loop

In [16]:
def train_dqn(env, agent, episodes=50, batch_size=32, target_update_freq=10):
    for ep in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            action = agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            agent.store((state, action, reward, next_state, done))
            agent.update(batch_size)
            state = next_state
            total_reward += reward

        agent.decay_epsilon()

        if ep % target_update_freq == 0:
            agent.update_target_network()

        print(f"Episode {ep+1}, Total Reward: {total_reward:.4f}, Epsilon: {agent.epsilon:.3f}")


In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix

def evaluate_dqn(env, agent, ticker, initial_capital=100000):
    state = env.reset()
    done = False
    portfolio = [env.balance]
    predictions = []
    actuals = []

    while not done:
        action = agent.select_action(state)
        next_state, reward, done, _ = env.step(action)

        # Store predicted and true label
        predictions.append(action)
        actual_label = env.df.loc[env.current_step, 'Label'] if env.current_step < len(env.df) else 0
        actuals.append(actual_label)

        portfolio.append(env.balance)
        state = next_state

    final_value = env.balance
    return_pct = (final_value - initial_capital) / initial_capital
    accuracy = accuracy_score(actuals, predictions)
    conf_matrix = confusion_matrix(actuals, predictions, labels=[-1, 0, 1])

    print(f"\n--- {ticker} [DQN] ---")
    print(f"Final Portfolio Value: ${final_value:,.2f}")
    print(f"Total Return: {return_pct:.2%}")
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:\n", conf_matrix)

    return {
        'final_portfolio_value': final_value,
        'return_pct': return_pct,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix,
        'portfolio_history': portfolio,
        'predictions': predictions,
        'actuals': actuals
    }



In [18]:
# Choose one symbol to train on
df = train_test_split['PLUG']['train'].copy()

# Environment and Agent setup
env = TradingEnv(df)
agent = DQNAgent(state_dim=4, action_dim=3)

# Train
train_dqn(env, agent)

# Evaluate on test data
test_env = TradingEnv(train_test_split['PLUG']['test'].copy())
evaluate_dqn(test_env, agent, ticker='PLUG')



/tmp/ipython-input-15-2004072197.py:51: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  states = torch.FloatTensor(states)


Episode 1, Total Reward: -0.5442, Epsilon: 0.995
Episode 2, Total Reward: 1.9512, Epsilon: 0.990
Episode 3, Total Reward: -1.4274, Epsilon: 0.985
Episode 4, Total Reward: 0.7206, Epsilon: 0.980
Episode 5, Total Reward: 0.5536, Epsilon: 0.975
Episode 6, Total Reward: 0.9986, Epsilon: 0.970
Episode 7, Total Reward: 1.1789, Epsilon: 0.966
Episode 8, Total Reward: 1.6830, Epsilon: 0.961
Episode 9, Total Reward: 0.6365, Epsilon: 0.956
Episode 10, Total Reward: 0.2846, Epsilon: 0.951
Episode 11, Total Reward: 1.6572, Epsilon: 0.946
Episode 12, Total Reward: 0.5723, Epsilon: 0.942
Episode 13, Total Reward: 2.0105, Epsilon: 0.937
Episode 14, Total Reward: 0.7157, Epsilon: 0.932
Episode 15, Total Reward: 1.0392, Epsilon: 0.928
Episode 16, Total Reward: -0.2172, Epsilon: 0.923
Episode 17, Total Reward: -0.0660, Epsilon: 0.918
Episode 18, Total Reward: 0.5443, Epsilon: 0.914
Episode 19, Total Reward: 0.4742, Epsilon: 0.909
Episode 20, Total Reward: 1.5972, Epsilon: 0.905
Episode 21, Total Reward:

{'final_portfolio_value': np.float64(198337.77928468536),
 'return_pct': np.float64(0.9833777928468536),
 'accuracy': 0.1750599520383693,
 'conf_matrix': array([[  0, 271, 158],
        [  0,  87,  38],
        [  0, 250, 132]]),
 'portfolio_history': [100000,
  100000,
  100000,
  np.float64(117901.23216045865),
  np.float64(117901.23216045865),
  np.float64(117901.23216045865),
  np.float64(117901.23216045865),
  np.float64(117901.23216045865),
  np.float64(117901.23216045865),
  np.float64(116745.33880824519),
  np.float64(116745.33880824519),
  np.float64(116745.33880824519),
  np.float64(115911.45091564277),
  np.float64(115911.45091564277),
  np.float64(115911.45091564277),
  np.float64(115911.45091564277),
  np.float64(104642.28047734866),
  np.float64(104642.28047734866),
  np.float64(104642.28047734866),
  np.float64(104642.28047734866),
  np.float64(104642.28047734866),
  np.float64(104642.28047734866),
  np.float64(104642.28047734866),
  np.float64(104642.28047734866),
  np.